## UBER Pickup - Clustering : Kmeans

Nous allons utiliser Kmeans, qui est un algorithme de clustering d'apprentissage non supervisé qui permet de créer des groupes de données en fonction de leur proximité géographique. 

In [187]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
import plotly
from plotly.graph_objs import Scattermapbox
import plotly.graph_objs as go
import plotly.io as pio
# pio.renderers.default = "svg"
pio.renderers.default = "iframe_connected"

In [188]:
merge_df = pd.read_csv('./src/datasets/merge_df.csv')

df_apr14 = pd.read_csv('./src/datasets/uber-raw-data-apr14_modified.csv')

In [189]:
merge_sample = merge_df.sample(500000)

On crée un dataset `test` pour faire des tests de clustering

In [136]:
df_test = df_apr14[(df_apr14['Date'] == '2014-04-01')]

In [137]:
coords = ['Lat','Lon']
X = df_test.loc[:,coords]

On scale les données pour faciliter les calculs

In [138]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [139]:
X

array([[ 0.7672978 ,  0.41505401],
       [-0.38027247, -1.10899284],
       [-0.2473388 , -0.20528667],
       ...,
       [-0.11711806, -0.30676215],
       [-0.70039845,  0.69076098],
       [ 0.23285018, -0.00425034]])

### KMEANS

L'algorithme `KMEANS` est un algorithme de clustering qui est un type d’apprentissage non supervisé

Il partitionne les données par un procédé itératif, cad qu'il va répéter une opération autant de fois que nécessaire avant de trouver le meilleur centroide (c'est la moyenne arithmétique de tous les points de données appartenant à un cluster particulier, c'est egalement un point du dataset que l’on choisit comme le « centre » d’un cluster).

* Il va d'abord initialiser un certain nombres de centroides dans un ensemble de données (défini à l'avance).

* Ensuite, il attribut chaque point de donnée au cluster le plus proche en essayant de maintenir les clusters les plus petits possibles et les autres clusters aussi différents que possible.

C'est un algorithme d'esperance-maximisation avec une matrice de covariance, qui utilise la distance euclidienne (directe entre 2 points)

Voici le deroulement de Kmeans:

* **On construit k clusters** : Chaque point est dans le cluster du centroïde qui lui est le plus proche.
* **On calcule les nouveaux centroïdes** : Pour chacun des clusters qu’on vient de former, on calcule la moyenne. Celle-ci devient le nouveau centroïde (n’est pas necessairement un point du jeu de donnée).
* **On recommence jusqu’à ce qu’à ce qu’il y ait convergence** : La convergence correspond au fait que les centroïdes ne changent pas après une mise à jour.

Faisons des tests pour voir le comportement de cet algorithme.

On commence par déterminer un nombre de centroides égale à 6, pour qu'il puisse séparer la zone de densité de clients en 6 groupes

In [140]:
km = KMeans(n_clusters=6)
km.fit(X)

KMeans(n_clusters=6)

In [141]:
df_test['km_clusters'] = km.labels_
df_test.groupby('km_clusters')[['km_clusters']].count().rename(columns={'km_clusters' : 'nb_per_cluster'})

,nb_per_cluster
km_clusters,
0,6105
1,402
2,1145
3,577
4,157
5,6160


In [142]:
fig = px.scatter_mapbox(data_frame=df_test, lat='Lat', lon='Lon', color='km_clusters', mapbox_style='carto-positron', zoom=10.2)
fig.show()

Pour savoir quel nombre de clusters sera le nombre idéal à choisir, il va falloir procéder à plusieurs essais. Nous allons d'abord tester sur plusieurs avec une affluence variable.

D'abord, nous pouvons considérer l'affluence qu'il y a en journée, puis considérer l'affluence qu'il y a en semaine. Nous allons créer plusieurs tranches horaires pour tester notre algorithme et utiliser la méthode Silhouette (avec le coude :elbow) pour déterminer les bonnes valeurs à donner ) KMEANS.

Commençons avec différents essais dans la même journée.

In [143]:
# on choisit la date du 1er avril 2014 pour faire nos tests
df_test = df_apr14[(df_apr14['Date'] == '2014-04-01')]

Nous allons créer plusieurs tranches horaires (6 tranches de 4h pour analyser le trafic)

In [144]:
df_test.head()

,Date/Time,Lat,Lon,Base,Date,month,day_name,day,hour
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,2014-04-01,4,Tuesday,1,0
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,2014-04-01,4,Tuesday,1,0
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,2014-04-01,4,Tuesday,1,0
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,2014-04-01,4,Tuesday,1,0
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,2014-04-01,4,Tuesday,1,0


In [145]:
def create_clusters(df , nb_clusters=6 , ranges=4):
    scaler = StandardScaler()

    coords = ['Lat','Lon']
    n,s,liste_dfs,liste_Xs = ranges,[],[],[]

    for i,el in enumerate([i for i in range(24)]):
        if i%n==0:
            s.append(el)

    for i,el in enumerate(s):
        globals() [f'df_test_{i+1}'] = df.loc[(df['hour'] >= el) & (df['hour'] < el+4), :]
        globals() [f'X_{i+1}'] = globals() [f'df_test_{i+1}'].loc[:,coords]
        globals() [f'X_{i+1}'] = scaler.fit_transform(globals() [f'X_{i+1}'])
        liste_dfs.append(globals() [f'df_test_{i+1}'])
        liste_Xs.append(globals() [f'X_{i+1}'])    
        
    km = KMeans(n_clusters=nb_clusters) # on determine un nombre fixe de clusters pour avoir un repère
    liste_df_kmeans, liste_df_groupby_clusters = [], []

    i=0
    while i<len(liste_dfs):
        for df_,X_ in zip(liste_dfs, liste_Xs):
            km.fit(X_)
            df_.reset_index(drop=True)
            df_['km_clusters'] = km.labels_
            globals() [f'df_kmeans_{i+1}'] = df_.groupby('km_clusters')[['km_clusters']].count().rename(columns={'km_clusters' : 'nb_per_cluster'})
            liste_df_kmeans.append(df_)
            liste_df_groupby_clusters.append(globals() [f'df_kmeans_{i+1}'])
            i+=1
            
    return liste_df_kmeans, liste_df_groupby_clusters, liste_Xs
        

In [146]:
liste_df_k, liste_groupby_c, Xs_ = create_clusters(df_test , nb_clusters=6 , ranges=4)

nous avons donc :</br>
* df_test_1 = tranche de  0h /  3h
* df_test_2 = tranche de  4h /  7h
* df_test_3 = tranche de  8h / 11h
* df_test_4 = tranche de 12h / 15h
* df_test_5 = tranche de 16h / 19h
* df_test_6 = tranche de 20h / 23h

Nous allons pouvoir observer les différences entre ces tranches horaires

In [147]:
def clusters_by_range_hours(df, liste_df_kmeans, mapbox_style):


    token = 'pk.eyJ1Ijoib3AzbjVlZCIsImEiOiJjbDllYjl6bGswaG9uM3NsOW0zaGJ4ZHVrIn0.Us-gSPz0QgMbKbPqGkDtjg'
    
    v = [True, False, False, False, False, False]

    fig = go.Figure()

    for i,df_ in enumerate(liste_df_kmeans):
        fig.add_trace(
            go.Scattermapbox(
                lat=df_['Lat'],
                lon=df_['Lon'],
                hovertext=df_['km_clusters'],
                hoverinfo='text',
                mode='markers',
                marker=dict(
#                    size=dict_df.loc[i, 'df_normalized'][dicts['criterion'][i]]*65,
                    color=df_['km_clusters'],
                    colorbar=dict(
                        title=dict(
                            side='right',
                            text='clusters',
                            font=dict(
                                color='black',
                                size=15,
                                family='Arial'
                                )
                            ),
                    #bgcolor='LightSkyBlue',
                    x=1.08,
                    y=0.5,
                    len=1.1)
                    ),
                visible=v[i]
                )
        )

    start = pd.to_datetime(df.loc[0,'Date']).strftime("%m/%d/%Y")
    end = pd.to_datetime(df.loc[554930,'Date']).strftime("%m/%d/%Y")

    fig.update_layout(mapbox_style=mapbox_style, 
                      mapbox_accesstoken=token,
                     title=dict(
                        text=f'Clusters de la ville de Chicago pour la période du {start} au {end}',
                        font=dict(
                            color='rgb(47, 138, 196)',
                            size=16,
                            family='Open Sans'
                        )
                    ))


    fig.update_mapboxes(
        bearing=0,
        center=dict(
            lat=40.7,
            lon=-74
        ),
        pitch=0,
        zoom=9)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            #bgcolor = "#4BE8E0",
            #bordercolor = "#4B9AC7",
            buttons = [
                    go.layout.updatemenu.Button(
                        label = "Tranche horaire 0h - 3h",
                        method = "update",
                        args = [{"visible" : [True, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 4h - 7h",
                            method = "update",
                            args = [{"visible" : [False, True, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 8h - 11h",
                            method = "update",
                            args = [{"visible" : [False, False, True, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 12h - 15h",
                            method = "update",
                            args = [{"visible" : [False, False, False, True, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 16h - 19h",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, True, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Tranche horaire 20h - 23h",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, True]}])
                ]
            )]
        )
  
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()
     

In [148]:
clusters_by_range_hours(df_test, liste_df_k, mapbox_style="carto-positron")

KMEANS est un algorithme de clusterisation qui fonctionne en paramétrant le nombre clusters que l'on souhaite obtenir(contrairement à DBSCAN qui détermine lui même le nombe de clusters)

Pour trouver ce nombre de clusters optimal K, cad des clusters qui représentent une certaine densité, nous avons le choix entre 2 techniques :

* **Elbow** : Vérifie si les points de données dans un cluster sont proches de leur centroide
* **Silhouette** : Vérifie si les clusters sont éloignés les uns des autres

En souhaitant trouver le nombre optimal de clusters, nous pourrions utiliser 2 paramètres : la somme des ecarts au carré et la distance moyenne de chaque point par rapport à son centroide :

* `Somme des écarts au carré par rapport de chaque observation par rapport à son centroïde d'un même cluster`: 
La somme des carrés intra-vluster est une mesure de la variabilité des observations au sein de chaque cluster. En général, un cluster qui a une petite somme de carrés est plus compact qu'un cluster qui a une grande somme de carrés. Les clusters qui ont des valeurs plus élevées présentent une plus grande variabilité des observations au sein du cluster.

* `Moyenne des distances entre les observations et le centroïde de chaque cluster` : La distance moyenne entre les observations et le centroïde du cluster est une mesure de la variabilité des observations au sein de chaque cluster. En général, un cluster qui a une distance moyenne plus petite est plus compact qu'un cluster qui a une distance moyenne plus grande. Les clusters qui ont des valeurs plus élevées présentent une plus grande variabilité des observations au sein du cluster.

Pour revenir sur nos 2 méthodes,  **Elbow** utilise `uniquement les distances intra-cluster` et **Silhouette** utilise une `combinaison de distances inter-cluster et intra-cluster`. la distance intra-cluster qu'utilise **Elbow** est indiqué dans le paramètre de Kmeans qui s'appelle **inertia**. Pour trouver un compromis à ces 2 différentes méthodes, on peut clusteriser à l'aide de l'inertie et évaluer à l'aide de **Silhouette**.

In [149]:
Xs_[1]

array([[ 0.34559532, -0.65201135],
       [-2.14966807,  0.43497132],
       [-0.37543022, -0.23083215],
       ...,
       [ 0.99966849,  0.44560716],
       [ 0.94559157,  0.44135283],
       [ 0.86576375,  0.39668231]])

Nous allons donc recupérer les valeurs de `KMeans().inertia_` qui correspond à la **distance intra-cluster** de la methode **Elbow** et faire le score de **Silhouette** que nous ploterons dans un graph pour observer les résultats (pour cela nous devons avoir le score de **Silhouette** en fonction des clusters)

Nous allons commencer par tranche de 15 itérations et ajuster (cad augmenter cette valeur) si besoin

In [150]:
def create_silhouette_score_by_range_hour(liste_X_scaled, n_iter=15):
    liste_inertias, liste_sils, liste_Ks = [],[],[]

    i,j=2,0
    while j < len(liste_X_scaled):
        K,sil,inertia = [],[],[]
        for i in range(2,n_iter):
            kmeans = KMeans(n_clusters=i, random_state=0) 
    #        print(j, i)
            kmeans.fit(liste_X_scaled[j])
            K.append(i) 
            inertia.append(kmeans.inertia_) 
            sil.append(silhouette_score(liste_X_scaled[j], kmeans.predict(liste_X_scaled[j])))  
        liste_sils.append(sil)
        liste_Ks.append(K)
        liste_inertias.append(inertia)
        j+=1
    return liste_sils, liste_Ks, liste_inertias

In [151]:
liste_sils, liste_Ks, liste_inertias = create_silhouette_score_by_range_hour(Xs_, n_iter=15)

In [152]:
def elbow_graph(liste_Ks, liste_sils):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=liste_Ks[0], y=liste_sils[0], mode='lines', name='lines'))

    for i in range(1,len(liste_Ks)):
        fig.add_trace(go.Scatter(x=liste_Ks[i], y=liste_sils[i], mode='lines', name='lines', visible = False))

    fig.update_layout(
            title = go.layout.Title(text = "Score de Elbow en fonction de tranches horaires", x = 0.5),
            showlegend = False)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            buttons = [
                        go.layout.updatemenu.Button(
                            label = "Tranche horaire 0h - 3h",
                            method = "update",
                            args = [{"visible" : [True, False, False, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 4h - 7h",
                                method = "update",
                                args = [{"visible" : [False, True, False, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 8h - 11h",
                                method = "update",
                                args = [{"visible" : [False, False, True, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 12h - 15h",
                                method = "update",
                                args = [{"visible" : [False, False, False, True, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 16h - 19h",
                                method = "update",
                                args = [{"visible" : [False, False, False, False, True,  False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 20h - 23h",
                                method = "update",
                                args = [{"visible" : [False, False, False, False, False, True]}])
                    ]
        )]
    )
    
    fig.show()

In [153]:
elbow_graph(liste_Ks, liste_sils)

Les zones de "coudes" sont entre 8 et 12 clusters donc nous pouvons prendre, si nous nous basons seulement sur elbow, un nombre de clusters dans cette tranche, cad en moyenne 10 clusters pour bien séparer la densité de clients.

In [154]:
def silhouette_graph(liste_Ks, liste_inertias):
    fig = go.Figure()

    fig.add_trace(go.Bar(x=liste_Ks[0], y=liste_inertias[0]))

    for i in range(1,len(liste_Ks)):
        fig.add_trace(go.Bar(x=liste_Ks[i], y=liste_inertias[i], visible = False))

    fig.update_layout(
            title = go.layout.Title(text = "Score de Silhouette en fonction de tranches horaires", x = 0.5),
            showlegend = False)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            buttons = [
                        go.layout.updatemenu.Button(
                            label = "Tranche horaire 0h - 3h",
                            method = "update",
                            args = [{"visible" : [True, False, False, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 4h - 7h",
                                method = "update",
                                args = [{"visible" : [False, True, False, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 8h - 11h",
                                method = "update",
                                args = [{"visible" : [False, False, True, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 12h - 15h",
                                method = "update",
                                args = [{"visible" : [False, False, False, True, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 16h - 19h",
                                method = "update",
                                args = [{"visible" : [False, False, False, False, True,  False]}]),
                        go.layout.updatemenu.Button(
                                label = "Tranche horaire 20h - 23h",
                                method = "update",
                                args = [{"visible" : [False, False, False, False, False, True]}])
                    ]
        )]
    )
    
    fig.show()

In [186]:
silhouette_graph(liste_Ks, liste_inertias)

#### Conclusion

On voit pour toutes les tranches horaires les mêmes représentations, cad qu'à partir d'un certain nombre de cluster(15) on ne voit plus de changement, donc nous pouvons prendre 15 clusters pour bien répartir les zones en fonction d'une certaine densité.

Nous devons, par contre, choisir un minimum de densité de nos clusters pour éviter aux chauffeurs Uber d'aller dans des zones de faibles regroupements de clients. Pour cela, nous allons calculer puis déterminer un nombre mininum de clients par zone pour afficher correctement ces clusters

Comme nous avions considérés les tranches horaires d'une même journée, nous allons maintenant étudier des clusters selon un jour de la semaine.

### 2 - Etude par 'day_name' pour déterminer le nombre optimal de clusters

In [156]:
df_sample2 = merge_df.sample(500000)
df_sample2.reset_index(drop=True, inplace=True)
df_sample2.head()

,Date/Time,Lat,Lon,Base,Date,month,day_name,day,hour
0,2014-04-25 23:13:00,40.6788,-73.9868,B02617,2014-04-25,4,Friday,25,23
1,2014-09-29 07:13:00,40.7734,-73.9893,B02764,2014-09-29,9,Monday,29,7
2,2014-06-20 20:13:00,40.7442,-73.9843,B02682,2014-06-20,6,Friday,20,20
3,2014-09-22 08:25:00,40.7740,-73.8724,B02682,2014-09-22,9,Monday,22,8
4,2014-08-25 16:04:00,40.7191,-73.9896,B02617,2014-08-25,8,Monday,25,16


In [157]:
df_groupby_sample1 = df_sample2.groupby('day_name')[['Date/Time']].count()
df_groupby_sample2 = df_sample2.groupby('day_name')[['Lat','Lon']].mean()
df_groupby_sample1.columns = ['count']
df_groupby_sample = pd.concat([df_groupby_sample1,df_groupby_sample2],axis=1)
df_groupby_sample

,count,Lat,Lon
day_name,,,
Friday,81797,40.739222,-73.974324
Monday,59568,40.739386,-73.969938
Saturday,71355,40.735683,-73.972719
Sunday,53458,40.733944,-73.966138
Thursday,83998,40.741064,-73.975331
Tuesday,73313,40.741630,-73.974731
Wednesday,76511,40.741474,-73.974996


On observe une tendance assez similaire en fonction des jours de la semaine sauf pour le Dimanche ou il y a moins de courses, ce qui est normal vu que c'est un jour ou les gens travaillent moins

Nous allons creer un dataset pour le mois d'avril uniquement en supprimant les clusters qui ne sont pas assez dense : nous allons pour cela créer un filtre qui permettra de pouvoir "mieux densifier" ces clusters

In [158]:
df_test2 = df_apr14.copy()

In [159]:
def create_clusters_by_dayname(df, nb_clusters=15):
    scaler = StandardScaler()

    coords = ['Lat','Lon']
    liste_dfs,liste_Xs = [],[]

    s = [day for day in df['day_name'].unique()]
    s[1],s[-2]  = s[-2],s[1]
    s[-2],s[-1] = s[-1],s[-2]

    for i,el in enumerate(s):
        globals() [f'df_test_{i+1}'] = df.loc[(df['day_name'] == el), :]
        globals() [f'X_{i+1}'] = globals() [f'df_test_{i+1}'].loc[:,coords]
        globals() [f'X_{i+1}'] = scaler.fit_transform(globals() [f'X_{i+1}'])
        liste_dfs.append(globals() [f'df_test_{i+1}'])
        liste_Xs.append(globals() [f'X_{i+1}'])    
        
    km = KMeans(n_clusters=nb_clusters) # on determine un nombre fixe de clusters pour avoir un repère
    liste_df_kmeans, liste_df_groupby_clusters = [], []

    i=0
    while i<len(liste_dfs):
        for df_,X_ in zip(liste_dfs, liste_Xs):
            km.fit(X_)
            df_.reset_index(drop=True)
            df_['km_clusters'] = km.labels_
            globals() [f'df_kmeans_{i+1}'] = df_.groupby('km_clusters')[['km_clusters']].count().rename(columns={'km_clusters' : 'nb_per_cluster'})
            liste_df_kmeans.append(df_)
            liste_df_groupby_clusters.append(globals() [f'df_kmeans_{i+1}'])
            i+=1
            
    return liste_df_kmeans, liste_df_groupby_clusters, liste_Xs
        

In [160]:
kmeans_dayname, groupbys_dayname , Xs_dayname = create_clusters_by_dayname(df_test2, nb_clusters=15)

In [161]:
def plot_clusters_by_dayname(df, liste_df_kmeans, mapbox_style):
    
    df = df.reset_index(drop=True)

    token = 'pk.eyJ1Ijoib3AzbjVlZCIsImEiOiJjbDllYjl6bGswaG9uM3NsOW0zaGJ4ZHVrIn0.Us-gSPz0QgMbKbPqGkDtjg'
    
    v = [True, False, False, False, False, False, False]

    fig = go.Figure()

    for i,df_ in enumerate(liste_df_kmeans):
        fig.add_trace(
            go.Scattermapbox(
                lat=df_['Lat'],
                lon=df_['Lon'],
                hovertext=df_['km_clusters'],
                hoverinfo='text',
                mode='markers',
                marker=dict(
#                    size=dict_df.loc[i, 'df_normalized'][dicts['criterion'][i]]*65,
                    color=df_['km_clusters'],
                    colorbar=dict(
                        title=dict(
                            side='right',
                            text='clusters',
                            font=dict(
                                color='black',
                                size=15,
                                family='Arial'
                                )
                            ),
                    #bgcolor='LightSkyBlue',
                    x=1.08,
                    y=0.5,
                    len=1.1)
                    ),
                visible=v[i]
                )
        )

    start = pd.to_datetime(df.loc[0,'Date']).strftime("%m/%d/%Y")
    end = pd.to_datetime(df.loc[len(df)-1,'Date']).strftime("%m/%d/%Y")

    fig.update_layout(mapbox_style=mapbox_style, 
                      mapbox_accesstoken=token,
                     title=dict(
                        text=f'Clusters de la ville de Chicago pour la période du {start} au {end}',
                        font=dict(
                            color='rgb(47, 138, 196)',
                            size=16,
                            family='Open Sans'
                        )
                    ))


    fig.update_mapboxes(
        bearing=0,
        center=dict(
            lat=40.7,
            lon=-74
        ),
        pitch=0,
        zoom=9)

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            #bgcolor = "#4BE8E0",
            #bordercolor = "#4B9AC7",
            buttons = [
                    go.layout.updatemenu.Button(
                        label = "Lundi",
                        method = "update",
                        args = [{"visible" : [True, False, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Mardi",
                            method = "update",
                            args = [{"visible" : [False, True, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Mercredi",
                            method = "update",
                            args = [{"visible" : [False, False, True, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Jeudi",
                            method = "update",
                            args = [{"visible" : [False, False, False, True, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Vendredi",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, True, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Samedi",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, True, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Dimanche",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, False, True]}])
                ]
            )]
        )
  
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()
     

In [162]:
plot_clusters_by_dayname(df_test2, kmeans_dayname, mapbox_style="carto-positron")

Ce test n'est pas concluant, car il est très difficile de voir une différence en fonction du jour de la semaine (en fonction d'une heure comme une heure de pointe par exemple, il y a une réelle différence avec une heure calme, comme la nuit) : Les cartes concernant les clusters par jour de la semaine sont decoupées en "quartier assez nette" ce qui laisse le choix aux chauffeurs Uber d'utiliser cet algorithme pour déterminer dans quelle zone il souhaite récupérer des clients.

Nous ne pouvons donc pas dire quel est le nombre de cluster optimal (15 quarties, 50 quartiers, 100 quartiers...) : l'énoncé nous impose un délai d'attente maximum de 7 minutes, mais nous ne pouvons pas connaitre, sans calcul de distance et donc de vitesse, le nombre de clusters optimal pour qu'un chauffeur Uber puisse récupérer un client en moins de 7 min après avoir déposé un ancien client (quel est l'intéret de découper en quartier de gros clusters si il y a toujours du monde dans cette zone)

De plus, nous ne connaissons pas la direction vers laquelle un client potentiel souhaite se déplacer : en prenant un cas extrème, comme celui qu'un client parte toujours dans la direction opposée à celle ou se trouve les gros clusters, il faudrait dans se cas inclure les tous petits clusters pour avoir une chance de récupérer un nouveau client en moins de 7 minutes, et il faudrait aussi déterminer un % de chance de partir dans la bonne direction, ce qui permettrai d'ajuster le nombre de clusters que l'on souhaite obtenir.

### 3 - Etude par regroupement de courses pour supprimer les clusters avec densité faible

Tout d'abord, nous allons créer un dataset qui prend un sample de 500k courses pour l'aggréger pouvoir determiner des clusters en fonction de moment .

Cela représente en gros 2 semaines de courses prises de manière aléatoire sur une période de 6 mois (2 semaines car 1/10 de 6 mois)

In [190]:
df_sample2 = merge_df.sample(500000)
df_sample2.reset_index(drop=True, inplace=True)
df_sample2.head()

,Date/Time,Lat,Lon,Base,Date,month,day_name,day,hour
0,2014-06-26 06:44:00,40.7514,-73.9739,B02598,2014-06-26,6,Thursday,26,6
1,2014-08-13 16:41:00,40.7477,-73.9749,B02617,2014-08-13,8,Wednesday,13,16
2,2014-06-08 16:42:00,40.7401,-73.9865,B02682,2014-06-08,6,Sunday,8,16
3,2014-07-23 18:54:00,40.7299,-73.9913,B02617,2014-07-23,7,Wednesday,23,18
4,2014-06-26 23:06:00,40.7516,-73.9837,B02617,2014-06-26,6,Thursday,26,23


In [191]:
scaler = StandardScaler()
coords = ['Lat','Lon']
X = df_sample2.loc[:,coords]
X = scaler.fit_transform(X)

N'oublions pas notre objectif qui est de trouver le nombre de clusters optimal :</br>
* Soit nous conservons le nombre de clusters actuel (15) qui représente les gros quartiers de New York
* Soit nous augmentons le nombre de clusters voulus (par exemple 50) en admettant l'hypothèse qu'un chauffeur Uber reste dans une zone assez restreinte, ce qui permettra un meilleur découpage des quartiers

En prenant en compte la 2e hypothèse, nous affirmons qu'il n'y a pas une saturation de chauffeur Uber dans une petite zone : cad qu'il y a assez de place dans un des 50 clusters pour que le chauffeur puisse y rester (car si cela n'etait pas vrai, il devrait changer de zone et donc le choix de prendre plus de clusters ne serait pas pertinent)

Testons donc avec un nombre de clusters égal à 50 et regardons le resultats (nous allons aussi comparer deux algorithmes de calcul de cluster : **`Algorithme espérance-maximisation ('lloyd')`** et **`Algorithme d'inégalité triangulaire('elkan')`**)

Sans rentrer en details dans ce notebook, car cela le rallongerai davantage, je vais faire une explication sommaire:</br>
* **Algorithme espérance-maximisation** : sépare les points de données en fonction d'une différence de distribution gaussienne
* **Algorithme d'inégalité triangulaire**: éviter le calcul des distances entre points de données qui sont redondantes( la longueur d'un côté est inférieure à la somme des longueurs des deux autres côtés : distance directe est la ligne droite)

Si vous voulez en savoir plus, je vous conseille ces liens qui sont très bien détaillé (tous les liens sont en francais):

**Algorithme espérance-maximisation**:</br>
* https://dridk.me/expectation-maximisation.html
* https://fr.wikipedia.org/wiki/Algorithme_esp%C3%A9rance-maximisation

**Algorithme d'inégalité triangulaire**:
* https://fr.wikipedia.org/wiki/In%C3%A9galit%C3%A9_triangulaire
* https://www.lri.fr/~sebag/Examens_2006/Elkan_resume.doc
* https://fr.wikipedia.org/wiki/Algorithme_de_Christofides

In [192]:
kmeans_lloyd = KMeans(n_clusters = 50, algorithm="full")
kmeans_lloyd.fit(X)

KMeans(algorithm='full', n_clusters=50)

In [193]:
kmeans_elkan = KMeans(n_clusters = 50, algorithm="elkan")
kmeans_elkan.fit(X)

KMeans(algorithm='elkan', n_clusters=50)

In [194]:
# numero du clusters avec l'algo esperance-maximisation
df_sample2['cluster_lloyd'] = kmeans_lloyd.labels_

# numero des clusters avec l'algo inégalité triangulaire
df_sample2['cluster_elkan'] = kmeans_elkan.labels_

In [195]:
df_sample2

,Date/Time,Lat,Lon,Base,Date,month,day_name,day,hour,cluster_lloyd,cluster_elkan
0,2014-06-26 06:44:00,40.7514,-73.9739,B02598,2014-06-26,6,Thursday,26,6,26,27
1,2014-08-13 16:41:00,40.7477,-73.9749,B02617,2014-08-13,8,Wednesday,13,16,26,27
2,2014-06-08 16:42:00,40.7401,-73.9865,B02682,2014-06-08,6,Sunday,8,16,1,5
3,2014-07-23 18:54:00,40.7299,-73.9913,B02617,2014-07-23,7,Wednesday,23,18,0,36
4,2014-06-26 23:06:00,40.7516,-73.9837,B02617,2014-06-26,6,Thursday,26,23,26,27
...,...,...,...,...,...,...,...,...,...,...,...
499995,2014-08-09 19:39:00,40.7577,-73.9779,B02682,2014-08-09,8,Saturday,9,19,26,27
499996,2014-06-03 21:51:00,40.7422,-74.0061,B02598,2014-06-03,6,Tuesday,3,21,19,37
499997,2014-08-30 04:42:00,40.7360,-73.9934,B02617,2014-08-30,8,Saturday,30,4,1,5
499998,2014-04-25 18:54:00,40.7718,-73.9675,B02617,2014-04-25,4,Friday,25,18,33,16


In [196]:
# regroupement des courses par jour
gb = df_sample2.groupby(['Date','month','day_name','day'])[['Date/Time']].count()
gb.columns = ['nb_courses_per_day']
gb

,,,,nb_courses_per_day
Date,month,day_name,day,
2014-04-01,4,Tuesday,1,1624
2014-04-02,4,Wednesday,2,1958
2014-04-03,4,Thursday,3,2251
2014-04-04,4,Friday,4,2929
2014-04-05,4,Saturday,5,2115
...,...,...,...,...
2014-09-26,9,Friday,26,4023
2014-09-27,9,Saturday,27,4320
2014-09-28,9,Sunday,28,3280


In [197]:
mean_by_day = gb['nb_courses_per_day'].mean()
mean_by_week = mean_by_day * 7
mean_by_week = int(mean_by_week)
mean_by_week

19125

Par semaine, en moyenne, il y 19125 courses

In [198]:
# regroupement des courses par heure
gb2 = df_sample2.groupby(['Date','month','day_name','day','hour'])[['Date/Time']].count()
gb2.columns = ['nb_courses_per_hour']
gb2

nb_courses_per_hour
Date       month day_name day hour                     
2014-04-01 4     Tuesday  1   0                      13
                              1                       5
                              2                       5
                              3                      15
                              4                      22
...                                                 ...
2014-09-30 9     Tuesday  30  18                    294
                              19                    269
                              20                    255
                              21                    211
                              22                    155

[4391 rows x 1 columns]

In [199]:
mean_by_hour = gb2['nb_courses_per_hour'].mean()
mean_by_hour = int(mean_by_hour)
mean_by_hour

113

Pour chaque course, donc pour un point sur la carte, on a attribué un numéro de cluster pour pouvoir les regrouper.

On va regrouper les nombres de clusters par date pour voir si nous devons en retirer pour eviter les zones à faible densité de clients

In [200]:
merge_df.shape

(4534327, 9)

In [201]:
clusters_lloyd = df_sample2.groupby('cluster_lloyd').count()['Date/Time'].values
                 
clusters_elkan = df_sample2.groupby('cluster_elkan').count()['Date/Time'].values

Nous allons donc garder que les clusters qui ont un nombre de points de données supérieur à un certain seuil : nous allons devoir définir ce seuil

Nous allons faire un bref calcul : nous souhaitons qu'un client n'attende que 5 min(pour anecdote, le temps d'un taxi jaune a NY est quasi nul car il y en a beaucoup). De plus, nous allons déterminer que le temps d'une course est égal au temps d'attente d'un client (pour simplifier les calculs) donc il faut 12 clients par heure.

Notre df_sample correspond à peu près à 14 jours de données : sachant que nous voulons minimum 12 clients / heure / cluster, il va falloir décomposer ces 14 jours en :

2 semaines = 14 jours = 14 * 24 heures

donc nous arrivons à : 14 * 24 * 12 clients = 4032 points minimum par cluster

In [202]:
nb_max_data_points = 14*24*12

# 2 semaines ==> 24h * 14j ==> 336h / 6 mois ==> 14h / semaine 
clusters_lloyd > nb_max_data_points

array([ True,  True,  True,  True, False, False, False,  True,  True,
       False, False,  True, False,  True, False, False, False, False,
       False,  True,  True,  True, False,  True, False,  True,  True,
       False, False, False,  True, False, False,  True, False,  True,
        True,  True, False, False, False, False, False,  True, False,
        True, False, False, False, False])

In [203]:
df_sample2['lloyd_is_dense'] = df_sample2['cluster_lloyd'].map(lambda row : 1 if clusters_lloyd[row]>nb_max_data_points else 0)
df_sample2['elkan_is_dense'] = df_sample2['cluster_elkan'].map(lambda row : 1 if clusters_elkan[row]>nb_max_data_points else 0)

In [204]:
df_sample2['lloyd_is_dense'].value_counts()

1    468577
0     31423
Name: lloyd_is_dense, dtype: int64

In [205]:
df_sample2['elkan_is_dense'].value_counts()

1    474869
0     25131
Name: elkan_is_dense, dtype: int64

In [206]:
df_sample2.to_csv('./src/datasets/df_sample2.csv', index=False)

In [207]:
df_sample2 = pd.read_csv('./src/datasets/df_sample2.csv')

In [208]:
fig = px.scatter_mapbox(df_sample2.loc[df_sample2['lloyd_is_dense']==1,:], lat='Lat', lon='Lon', color='cluster_lloyd', zoom=10, mapbox_style='carto-positron')
fig.show()

On observe que les clusters qui sont vers l'extérieur sont plus étendus(moins dense) que les clusters centraux. Nous pourrions donc améliorer l'algorithme en y ajoutant des contraintes (des poids) sur le prix en fonction de l'eloignement du centre.

In [209]:
test = df_sample2.sample(50000)

In [210]:
df_filter_lloyd = test.loc[test['lloyd_is_dense']==1,:]
df_filter_elkan = test.loc[test['elkan_is_dense']==1,:]

dic = {
    'df' : [df_filter_lloyd, test, df_filter_elkan, test], 
    'color' : ['cluster_lloyd', 'cluster_lloyd', 'cluster_elkan', 'cluster_elkan']
    }

In [211]:
def plot_lloyd_elkan_algo_cluster(df, dic):
    mapbox_access_token = 'pk.eyJ1Ijoib3AzbjVlZCIsImEiOiJjbDllYjl6bGswaG9uM3NsOW0zaGJ4ZHVrIn0.Us-gSPz0QgMbKbPqGkDtjg'

    v = [True, False, False, False]


    fig = go.Figure()

    for df, color in zip(dic['df'],dic['color']):

        fig.add_trace(go.Scattermapbox(
                lat = df.loc[:, 'Lat'],
                lon = df.loc[:, 'Lon'],
                mode = 'markers',
                marker = dict(
                    color=df[color],
                    size = 6,
                    opacity = 0.7,
                    colorbar = dict(
                        titleside = "right",
                        outlinecolor = "rgba(68, 68, 68, 0)"
                    ),
                ),
                text=[color],
            )) 

    fig.update_layout(
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=40.73,
                lon=-73.9
            ),
            pitch=0,
            zoom=9
        )
    )

    fig.update_layout(
            updatemenus = [go.layout.Updatemenu(
                active = 0,
                #bgcolor = "#4BE8E0",
                #bordercolor = "#4B9AC7",
                buttons = [
                        go.layout.updatemenu.Button(
                            label = "Cluster Lloyd Dense",
                            method = "update",
                            args = [{"visible" : [True, False, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Cluster Lloyd Normal",
                                method = "update",
                                args = [{"visible" : [False, True, False, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Cluster Elkan Dense",
                                method = "update",
                                args = [{"visible" : [False, False, True, False]}]),
                        go.layout.updatemenu.Button(
                                label = "Cluster Elkan Normal",
                                method = "update",
                                args = [{"visible" : [False, False, False, True]}])
                    ]
                )]
            )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()

In [212]:
plot_lloyd_elkan_algo_cluster(test, dic)

On observe bien une différence entre les clusters Dense (qui ont un nombre minimal de client de 12 par heure) et ceux qui sont plus éparpillés.

Il reste cependant des imperfections, cad des zones dans lesquelles il sera difficile de trouver des clients qui, de plus, sont très éloignés des zones à fortes densités.

In [ ]:
def formating_df_kmeans(df, kmean_clusters=12):
    """Permet de créer une colonne 'nombre de clusters' par heure qui est le nombre de course associé à son cluster"""

    df_ = pd.DataFrame()

    months = df.loc[:,'month'].unique().tolist()
    days = df.loc[:,'day'].unique().tolist()
    hours = df.loc[:,'hour'].unique().tolist()
    error_set = []

    for month in months:
        for day in days:
            for hour in hours:
                # On crée un dataframe temporaire qui correspond au mois, au jour et à l'heure donnée
                df_temp = df[(df['month'] == month) & (df['day'] == day) & (df['hour'] == hour)]
                
                # On selectionne les coordonnées pour configurer DBSCAN
                coords = df_temp.loc[:, ['Lat','Lon']]
                kmeans = KMeans(n_clusters=kmean_clusters)
                try:
                    km = kmeans.fit(coords)
                    df_temp['clusters'] = kmeans.labels_
                    # le nombre de clusters est obtenu en regroupant les clusters par heure
                    nb_clusters = df_temp.groupby('clusters')['hour'].count()
                    nb_clusters.name = 'nb_clusters'
                    # si on a un index negatif, on le remet à 0
                    if nb_clusters.index[0] == -1:
                        nb_clusters.index += 1 
                    # on crée un centre des coordonnées en regroupant par clusters et en prenant la moyenne des coords
                    centers_coords = df_temp.groupby('clusters')[['Lat','Lon']].mean().reset_index()
                    n = centers_coords.shape[0]
                except:
                    pass 
                
                h = pd.Series([hour]*n)
                h.name = 'hour'
                # on concat en colonne les coords gps, le nombre de clusters et les heures
                df_temp = pd.concat([centers_coords,nb_clusters,h],axis=1)
                df_temp['month'] = month
                df_temp['day'] = day
                df_temp['hour'] = hour
                try:
                    df_temp["Date/Time"] = pd.to_datetime(df_temp['month'].astype(str) +"-"+ df_temp["day"].astype(str)+"-2014")
                    df_temp["day_name"] = df_temp['Date/Time'].dt.day_name()
                except:
                    error_set.append(f"Error date for {month}-{day}")
                
                # on concat en ligne le dataframe d'origine avec celui obtenu
                df_ = pd.concat([df_,df_temp],axis=0)
                
    df_ = df_.reset_index().drop(['index'],axis=1)   
    df_ = df_.dropna()
    error_set = np.unique(error_set)
    print(error_set)
    print(df_['clusters'].unique())
    
    return df_

In [57]:
kmeans = formating_df_kmeans(merge_df, kmean_clusters=12)

['Error date for 4-31' 'Error date for 6-31' 'Error date for 9-31']
[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [58]:
kmeans.tail(8)

,clusters,Lat,Lon,nb_clusters,hour,month,day,Date/Time,day_name
53272,4,40.347900,-74.654600,1,23,9,30,2014-09-30,Tuesday
53273,5,40.709377,-74.186346,13,23,9,30,2014-09-30,Tuesday
53274,6,40.851840,-73.916860,25,23,9,30,2014-09-30,Tuesday
53275,7,40.745483,-73.571567,6,23,9,30,2014-09-30,Tuesday
53276,8,40.764641,-73.880308,51,23,9,30,2014-09-30,Tuesday
53277,9,40.712762,-73.946510,122,23,9,30,2014-09-30,Tuesday
53278,10,40.942150,-74.308850,2,23,9,30,2014-09-30,Tuesday
53279,11,40.719640,-73.999602,284,23,9,30,2014-09-30,Tuesday


In [8]:
kmeans.to_csv('./src/datasets/kmeans.csv', index=False)

In [9]:
kmeans = pd.read_csv('./src/datasets/kmeans.csv')

In [14]:
def filtered_df(df, month=None, day=None, day_name=None, hour=None):  
    return df[(df['month'] == month) & ((df['day'] == day) | (df['day_name'] == day_name)) & (df['hour'] == hour)]

In [34]:
def clustering(df, month=None, day=None, day_name=None, hour=None):
    if not day_name:
        day_name = df[(df['month'] == month) & (df['day'] == day)]['day_name'].values[0]
    df_test = filtered_df(df, month, day, day_name, hour)
    clusters_df = df_test.groupby('clusters')['Lat','Lon'].mean()
    clusters_df['total_people_per_month'] = df.groupby('clusters')['clusters'].count()
    clusters_df['%_people_per_cluster'] = round(clusters_df['total_people_per_month'] / len(df)*100,2)
    if day_name:
        clusters_df[f'mean_people_per_{day_name}'] = clusters_df['total_people_per_month'].apply(lambda x: x//len(df['day'].unique()))
    clusters_df['mean_people_per_month'] = clusters_df['total_people_per_month'].apply(lambda x: x//df['day'].nunique())
    clusters_df['%_people_per_month'] = clusters_df['mean_people_per_month'] / len(df)
    clusters_df['month'] = month
    clusters_df['day_name'] = day_name
    clusters_df['day'] = day
    clusters_df['hour'] = hour
    if any(clusters_df['day'].isna()):
        clusters_df.drop(['day'],axis=1,inplace=True)
        
    clusters_df = clusters_df.reset_index()
    
    return clusters_df

In [35]:
clustering(kmeans, month=4, day=1, day_name=None, hour=2)

,clusters,Lat,Lon,total_people_per_month,%_people_per_cluster,mean_people_per_Tuesday,mean_people_per_month,%_people_per_month,month,day_name,day,hour
0,0,40.699434,-73.975267,4392,8.33,141,141,0.002675,4,Tuesday,1,2
1,1,40.816598,-73.909763,4392,8.33,141,141,0.002675,4,Tuesday,1,2
2,2,40.849802,-73.860028,4392,8.33,141,141,0.002675,4,Tuesday,1,2
3,3,40.701013,-73.917935,4392,8.33,141,141,0.002675,4,Tuesday,1,2
4,4,40.796192,-73.945758,4392,8.33,141,141,0.002675,4,Tuesday,1,2
5,5,40.792283,-73.953000,4392,8.33,141,141,0.002675,4,Tuesday,1,2
6,6,40.728679,-73.968833,4392,8.33,141,141,0.002675,4,Tuesday,1,2
7,7,40.703710,-73.952861,4392,8.33,141,141,0.002675,4,Tuesday,1,2
8,8,40.728192,-73.932641,4392,8.33,141,141,0.002675,4,Tuesday,1,2
9,9,40.747905,-73.947745,4392,8.33,141,141,0.002675,4,Tuesday,1,2


**CONCLUSION** :
    
L'algorithme KMEANS arrive bien à créer plusieurs clusters d'une taille similaire pour aider les chauffeurs Uber à trouver de petites zones dans lesquelles ils pourront récupérer des clients et faire de petits trajets, ce qui leur evitera de perdre beaucoup d'essence et de temps pour trouver des clients.

La contrepartie de cet algorithme est que si nous avons beaucoup de chauffeurs dans une même zone, et qu'il se trouve dans des zones à faible densité ou des zones à bordures d'autres zones, il va alors être difficile de savoir vers quelle direction aller.

Enfin, la manière dont KMEANS fonctionne pour la clusterisation des clients est qu'il va créer des cercles plutôt que des découpages de secteurs géographiques, ce qui peut poser problème pour la séparation par quartier des clients. Le fait que des zones à faibles densités persistent, après la selection d'un nombre minimum de client pour une heure donnée, nous oblige à changer de stratégie pour ces zones : devons nous faire des prix différents en fonction de la densité de population de certaines zones (ce qui serait injuste pour ces clients de ne pas habiter dans des endroits plus dense) ?

Cela risque donc de délaisser certaines zones et pénaliser les clients qui seraient entre ces zones car ils ont été filtrés par densité.